In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import copy

import pandas as pd
from datasets import Dataset, DatasetDict

from detection_datasets import Converter

In [ ]:
read_config = {
    'dataset_format': 'coco',
    'path': '/Users/jeromeblin/self/blinjrm/detection-datasets/data/raw',
    'splits': {
        'train': ('train.json', 'train'),
        'test': ('test.json', 'test'),
    },
}

converter = Converter()
converter.read(**read_config)

In [ ]:
converter.data.head()

In [ ]:
ds = converter.dataset.to_hf_dataset()

In [ ]:
ds

In [ ]:
ds['train'][0]

In [ ]:
df_splits = []
for key in ds.keys():  
    df = ds[key].to_pandas()
    df['split'] = key
    df_splits.append(df)

final_df = pd.concat(df_splits)
final_df

In [ ]:
objects = pd.json_normalize(final_df['objects'])
objects

In [ ]:
final_df_normalized = final_df.join(objects)
final_df_normalized

In [ ]:
def hf_datasetdict_to_pandas(ds: DatasetDict, explode: bool=False) -> pd.DataFrame:
    df_splits = []
    for key in ds.keys():  
        df_split = ds[key].to_pandas()
        df_split['split'] = key
        df_splits.append(df_split)

    df = pd.concat(df_splits)
    
    objects = pd.json_normalize(df['objects'])
    objects = objects.rename(columns={'id': 'bbox_id'})
    
    final_df = df.join(objects)
    final_df = final_df.drop(columns=['objects'])
    
    if explode:
        return final_df.explode(["bbox_id", "category", "area", "bbox"]).set_index(["image_id", "bbox_id"])
    
    return final_df

In [ ]:
hf_datasetdict_to_pandas(ds=ds)

In [ ]:
hf_datasetdict_to_pandas(ds=ds, explode=True)